In [ ]:
import praw
import csv
import time
import os
from datetime import datetime
from dotenv import load_dotenv

# Load variabel dari .env
load_dotenv()

# Inisialisasi objek Reddit dengan kredensial dari .env
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT"),
)

# Ambil 10 postingan dengan query "pemerintah" yang disortir berdasarkan hot
subreddit = reddit.subreddit("all")
posts = subreddit.search("pemerintah", sort="hot", limit=10)

# List untuk menyimpan komentar (id, body, dan waktu upload) dengan minimal 5 kata
all_comments = []

def extract_comments(comment):
    if isinstance(comment, praw.models.MoreComments):
        return
    if len(comment.body.split()) >= 5:
        waktu = datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        all_comments.append([comment.id, comment.body, waktu])
    for reply in comment.replies:
        extract_comments(reply)

# Lakukan ekstraksi komentar untuk tiap posting
for post in posts:
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        extract_comments(comment)
    time.sleep(2)  # delay agar tidak terkena rate limit

print(f"Total komentar (>= 5 kata) yang diambil: {len(all_comments)}")

# Simpan hasil ke CSV
csv_filename = "reddit_comments.csv"
with open(csv_filename, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
    writer.writerow(["id", "komentar", "waktu"])
    writer.writerows(all_comments)

print(f"File CSV berhasil disimpan dengan nama: {csv_filename}")

RequestException: error with request HTTPSConnectionPool(host='www.reddit.com', port=443): Max retries exceeded with url: /api/v1/access_token (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000020E29FE2450>, 'Connection to www.reddit.com timed out. (connect timeout=16.0)'))